# Trading Pipeline Demo

This notebook demonstrates how to run the trading pipeline with multiple agents to generate a strategy for a given ticker.

In [ ]:
from trading_bot.agents import (
    TechnicalAnalysisAgent, MarketScannerAgent,
    SocialMediaAgent, NewsAnalyzerAgent,
)
from trading_bot.pipeline import Pipeline
from trading_bot.storage import JSONStorage
from trading_bot.portfolio import Portfolio


In [ ]:
agents = [TechnicalAnalysisAgent(), MarketScannerAgent(),
          SocialMediaAgent(), NewsAnalyzerAgent()]
pipeline = Pipeline(agents, storage=JSONStorage("data"),
                    portfolio=Portfolio())
result = pipeline.run_for_symbol("TSLA")
result["strategy"]
